In [7]:
import pandas as pd
import json
import fasttext
import emoji
import re
import collections
from urllib.parse import urlparse

# Loading merged data from channels

In [8]:
df_channels = pd.read_csv('/Users/katerynaburovova/PycharmProjects/dehumanization/data/merged_dataset/df_channels.csv', header=None)

/var/folders/2w/hygxgpp5353_dy_tk88gw2100000gn/T/ipykernel_36611/2069067117.py:1: DtypeWarning: Columns (0,1,3,4,6,9,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_channels = pd.read_csv('/Users/katerynaburovova/PycharmProjects/dehumanization/data/merged_dataset/df_channels.csv', header=None)


In [9]:
df_channels.columns = df_channels.iloc[0]
df_channels = df_channels[1:]

In [10]:
df_channels = (df_channels
               .drop(columns=['Unnamed: 0'], axis=1)
               .reset_index(drop=True))

In [11]:
len(df_channels)

8108693

In [12]:
df_channels.sample(5)

,id,date,views,reactions,to_id,fwd_from,message,type,duration,channel_name,frw_from_title,frw_from_name,msg_entity
3748791,8690.0,2022-07-05 17:55:03+00:00,NaN,MessageReactions(results=[ReactionCount(reacti...,PeerChannel(channel_id=1661202164),NaN,🙊,text,NaN,moscowach,NaN,NaN,NaN
7516803,11236.0,2022-12-05 09:32:40+00:00,14657.0,MessageReactions(results=[ReactionCount(reacti...,PeerChannel(channel_id=1169436645),"MessageFwdHeader(date=datetime.datetime(2022, ...",#внеэфира По поводу снимков вспомнилась давняя...,photo,NaN,grishkafilippov,NaN,NaN,NaN
603322,17430.0,2021-04-28 08:32:01+00:00,48847.0,NaN,PeerChannel(channel_id=1223219553),NaN,В деле бывшего руководителя девелоперской груп...,photo,NaN,vchkogpu,NaN,NaN,NaN
3598705,5944.0,2020-11-15 08:27:32+00:00,17572.0,NaN,PeerChannel(channel_id=1263953596),"MessageFwdHeader(date=datetime.datetime(2020, ...",Развернувшаяся среди коллег крайне важная диск...,text,NaN,grey_zone,Андрей Медведев,MedvedevVesti,[<telethon.tl.types.MessageEntityUrl object at...
6107474,34271.0,2021-08-26 06:10:01+00:00,16071.0,NaN,PeerChannel(channel_id=1099860397),NaN,Власти Москвы выделили 1 млрд руб. для компенс...,text,NaN,rbc_news,NaN,NaN,NaN


## We drop na for now (needs some mending)

In [14]:
df_channels = df_channels[~df_channels['message'].isna()]

In [15]:
f"We have {len(df_channels[(df_channels['message']=='nan')&(df_channels['type']=='text')].channel_name.unique())/len(df_channels):.4%} nans"

'We have 0.0000% nans'

### Loading list of unique channel name handles referenced by the group

In [16]:
names = (df_channels['frw_from_name']
              .value_counts()
              .reset_index(name="count")[1:]['index']
              .to_list())
len(names)

1959

In [17]:
len(names)

1959

In [18]:
dict_names = {"titles": names}
json_object = json.dumps(dict_names, indent=4)
with open("/Users/katerynaburovova/PycharmProjects/dehumanization/data/names_channels_list.json", "w") as outfile:
    outfile.write(json_object)

# Exploring the data

In [19]:
class LanguageIdentification:

    def __init__(self):
        pretrained_lang_model = "/Users/katerynaburovova/PycharmProjects/comp_soc_sci_projects/fasttext/lid.176.bin"
        self.model = fasttext.load_model(pretrained_lang_model)

    def predict_lang(self, text, label_only=True):
        predictions = self.model.predict(text, k=1)
        if label_only:
            return predictions[0][0][-2:]
        return predictions

In [20]:
lang_identifier = LanguageIdentification()
df_channels['message'] = df_channels['message'].apply(lambda x: str(x).replace('\n', ' '))
df_channels['lang'] = df_channels['message'].apply(lambda x: lang_identifier.predict_lang(x))

/var/folders/2w/hygxgpp5353_dy_tk88gw2100000gn/T/ipykernel_36611/324986298.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_channels['message'] = df_channels['message'].apply(lambda x: str(x).replace('\n', ' '))
/var/folders/2w/hygxgpp5353_dy_tk88gw2100000gn/T/ipykernel_36611/324986298.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_channels['lang'] = df_channels['message'].apply(lambda x: lang_identifier.predict_lang(x))


In [21]:
df_channels['lang'].value_counts()

ru    6901187
en      48321
uk      21941
bg       5814
de       5520
       ...   
mf          1
an          1
nb          1
iq          1
ep          1
Name: lang, Length: 138, dtype: int64

# Text preprocesing

## Separating comments from posts

In [22]:
df_channels[df_channels['channel_name']=='Topaz_Govorit']

,id,date,views,reactions,to_id,fwd_from,message,type,duration,channel_name,frw_from_title,frw_from_name,msg_entity,lang
5847662,2762.0,2022-12-02 18:01:03+00:00,8538.0,MessageReactions(results=[ReactionCount(reacti...,PeerChannel(channel_id=1321128351),NaN,"Помнится, что за месяц до начала войны с Софий...",text,NaN,Topaz_Govorit,NaN,NaN,NaN,ru
5847663,980465.0,2022-12-02 18:01:55+00:00,NaN,NaN,PeerChannel(channel_id=1679205140),NaN,На бумаге никакой войны нет,text,NaN,Topaz_Govorit,NaN,NaN,NaN,ru
5847664,980466.0,2022-12-02 18:02:11+00:00,NaN,MessageReactions(results=[ReactionCount(reacti...,PeerChannel(channel_id=1679205140),NaN,Так это закономерность ВС РФ демонстрируют не ...,text,NaN,Topaz_Govorit,NaN,NaN,NaN,ru
5847665,980467.0,2022-12-02 18:02:31+00:00,NaN,NaN,PeerChannel(channel_id=1679205140),NaN,и при всём при этом униатская церковь всё ещё ...,text,NaN,Topaz_Govorit,NaN,NaN,NaN,ru
5847666,980468.0,2022-12-02 18:02:37+00:00,NaN,NaN,PeerChannel(channel_id=1679205140),NaN,Интересно что скажет патриарх.,text,NaN,Topaz_Govorit,NaN,NaN,NaN,ru
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5860896,10.0,2019-07-29 16:08:49+00:00,1793.0,MessageReactions(results=[ReactionCount(reacti...,PeerChannel(channel_id=1321128351),NaN,Наверное каждый на своём пути встречал такого ...,photo,NaN,Topaz_Govorit,NaN,NaN,NaN,ru
5860898,8.0,2019-07-29 15:31:10+00:00,1727.0,MessageReactions(results=[ReactionCount(reacti...,PeerChannel(channel_id=1321128351),NaN,А ещё иногда будут смешные для меня мемы собст...,text,NaN,Topaz_Govorit,NaN,NaN,NaN,ru
5860899,7.0,2019-07-29 15:15:42+00:00,1924.0,MessageReactions(results=[ReactionCount(reacti...,PeerChannel(channel_id=1321128351),NaN,"Если хотите изменить мир вокруг себя, не нужно...",text,NaN,Topaz_Govorit,NaN,NaN,NaN,ru
5860900,6.0,2019-07-29 14:20:06+00:00,1935.0,MessageReactions(results=[ReactionCount(reacti...,PeerChannel(channel_id=1321128351),NaN,Интересную вещь заметил: каждый раз когда в МС...,text,NaN,Topaz_Govorit,NaN,NaN,NaN,ru


In [23]:
id_list = df_channels[['channel_name', 'to_id']].drop_duplicates(keep='first').drop_duplicates(subset=['channel_name'], keep='first').to_id.tolist()

In [24]:
df_channels['is_post'] = df_channels.apply(lambda x: True if x.to_id in id_list else False, axis=1)

/var/folders/2w/hygxgpp5353_dy_tk88gw2100000gn/T/ipykernel_36611/1862213152.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_channels['is_post'] = df_channels.apply(lambda x: True if x.to_id in id_list else False, axis=1)


In [25]:
print(f'{len(df_channels[df_channels["is_post"]==True])/len(df_channels)*100:.1f}% of messages are posts, the rest are comments')

97.4% of messages are posts, the rest are comments


In [26]:
df_comments = df_channels[df_channels['is_post']==False]

In [27]:
df_channels = df_channels[df_channels['is_post']==True]

In [28]:
df_channels.channel_name.unique()

array(['mardanaka', 'rian_ru', 'tvrain', 'krispotupchik', 'akimapachev',
       'go338', 'KotNaMirotvorze', 'emphasises', 'russ_orientalist',
       'pravda_shuravi', 'voenacher', 'n_zackhaim', 'Hinshtein',
       'kashinguru', 'lentachold', 'wargonzo', 'foxandraven',
       'madam_secretar', 'russianfuture', 'vchkogpu', 'rusich_army',
       'botcharov', 'mashmoyka', 'vv_volodin', 'Mikle1On',
       'kremlinprachka', 'ctrs2018', 'mediazzzona', 'sotaproject',
       'znachit_net', 'er_molnia', 'razvedkavperedZ', 'rasstrelny',
       'govoritfursov', 'gramotyyaroslava', 'informnapalm',
       'podosokorsky', 'mosnow', 'daokedao', 'chtddd', 'nevzorovtv',
       'SonOfMonarchy', 'lesyaryabtseva', 'fontankaspb', 'tass_agency',
       'ErnestV_2020', 'sorok40russia', 'odinokayakoko', 'pgubarev',
       'swodki', 'meduzalive', 'Gori_spb', 'anna_news', 'readovkanews',
       'umar_kremlev', 'government_rus', 'Alekhin_Telega', 'leylinurimm',
       'pushilindenis', 'strelets_molodec', 'istrkal

## Removing breaks and repr symbols

In [24]:
test_string = df_channels[df_channels['is_post']==True].message[0]
test_string

'А трансляция тем временем идет. Подписывайтесь  https://www.youtube.com/watch?v=4L7T3u7utSw'

In [25]:
splitters = ['\n', '\t', '\\n','\xa0', '\u200b']

In [26]:
df_channels['message_no_breaks'] = df_channels['message'].str.replace('\n|\t|\\n', ' ', case=False)

/var/folders/2w/hygxgpp5353_dy_tk88gw2100000gn/T/ipykernel_56821/3913600352.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_channels['message_no_breaks'] = df_channels['message'].str.replace('\n|\t|\\n', ' ', case=False)


## Isolating emojis

In [27]:
def get_emoji_count(text):
    return collections.Counter([match["emoji"] for word in text for match in emoji.emoji_list(word)])

In [28]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r' ', string)

In [29]:
%%time
df_channels['emojis'] = df_channels['message'].apply(lambda x: get_emoji_count(x))

CPU times: user 3min 42s, sys: 1.77 s, total: 3min 44s
Wall time: 8min 9s


In [30]:
%%time
df_channels['message_no_emoji'] = df_channels['message_no_breaks'].apply(lambda x: remove_emoji(x))

CPU times: user 7.78 s, sys: 189 ms, total: 7.97 s
Wall time: 8.03 s


In [31]:
df_channels['message_no_emoji'].sample(10).iloc[0]

'Сенсационное научное открытие!'

## Isolating urls

In [32]:
def find_urls(text):
    try:
        links =  re.findall(r'(https?://[^\s]+)', text)
        return [urlparse(item).netloc for item in links]
    except ValueError:
        return []

In [33]:
df_channels['url_list'] = df_channels['message_no_emoji'].apply(lambda x: find_urls(x))

In [34]:
df_channels['message_no_urls'] = df_channels['message_no_emoji'].str.replace('http\S+|www.\S+', '', case=False)

/var/folders/2w/hygxgpp5353_dy_tk88gw2100000gn/T/ipykernel_56821/343568952.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_channels['message_no_urls'] = df_channels['message_no_emoji'].str.replace('http\S+|www.\S+', '', case=False)


In [35]:
#bs check
(df_channels['message_no_urls']
 .apply(lambda x: find_urls(x))
 .value_counts())

[]    1226517
Name: message_no_urls, dtype: int64

In [36]:
df_channels['message_no_urls'].sample(10).iloc[0]

'  Власти Великобритании одобрили ужесточение санкций против России  Власти Великобритании одобрили поправки к закону о санкциях в отношении России. Теперь власти Великобритании по своему усмотрению смогут ужесточить ранее наложенные ограничительные меры.  Поправки к закону допускают санкции против широкого круга лиц и компаний из РФ, которые, по мнению британских властей, причастны к дестабилизации обстановки у границы Украины.'

## Extract mentions

In [37]:
df_channels['mentions'] = df_channels['message_no_urls'].str.findall(r"@([a-zA-Z0-9_]{1,50})")

In [38]:
df_channels[['mentions','message_no_urls']].sample(10)
# df_channels[['mentions','message_no_urls']].message_no_urls.iloc[1265478]

,mentions,message_no_urls
397948,[],"Основатель Pink Floyd Роджер Уотерс заявил, чт..."
1185708,[],"«Если НАТО признает, что наш самый большой стр..."
1267463,[kstati_p],Интересная получается картина - Любовь Соболь ...
1456731,[izvestia],Советский хоккеист и тренер Вячеслав Старшино...
644824,[],"«В Германии, Бельгии, Чехии, Дании нет таког..."
609666,[],​​Больше не президент Президент Татарстана Ми...
1001016,[glavpolit],Председатель правительства России Михаил Мишус...
1169556,[],"Нацисты бегут из Красного Лимана, украинские ..."
1078016,[],В Израиле объяснили тайный визит Нетаньяху в...
715297,[],Просто нардеп Парубий на блокпосту под Киевом....


In [39]:
# text_tst = df_channels[['mentions','message_no_urls']].message_no_urls.iloc[1265478]
# m_list = df_channels[['mentions','message_no_urls']].mentions.iloc[1265478]

In [40]:
def cut_mentions(list_of_mentions, text):
    for word in list_of_mentions:
        wrd = '@' + word
        text = text.replace(wrd, '')
    return text.strip()

In [41]:
%%time
df_channels['message_no_mentions'] = df_channels.apply(lambda x: cut_mentions(x['mentions'], x['message_no_urls']), axis=1)

CPU times: user 5.02 s, sys: 2.56 s, total: 7.57 s
Wall time: 8.4 s


In [42]:
mentions_col_list = df_channels[df_channels['mentions'].map(lambda d: len(d)) > 0]['mentions'].tolist()
mentions_list = [item for sublist in mentions_col_list for item in sublist]
mentions_unique = list(set(mentions_list))

In [43]:
print(f'We have {len(mentions_unique)} unique channel names among {len(mentions_list)} mentions for dataset')

We have 9872 unique channel names among 288370 mentions for dataset


In [44]:
dict = {"names": mentions_list}
json_object = json.dumps(dict, indent=4)
with open("/Users/katerynaburovova/PycharmProjects/dehumanization/data/names_from_mentions.json", "w") as outfile:
    outfile.write(json_object)

## Unify and isolate quotes

In [45]:
to_replace = ["«", "»", "“"]

df_channels['message_no_mentions'] = df_channels['message_no_mentions'].str.replace('|'.join(to_replace),'"')

/var/folders/2w/hygxgpp5353_dy_tk88gw2100000gn/T/ipykernel_56821/4136279409.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_channels['message_no_mentions'] = df_channels['message_no_mentions'].str.replace('|'.join(to_replace),'"')


In [46]:
%%time
df_channels['quotes'] = df_channels['message_no_mentions'].str.findall(r'"(.*?)"')

CPU times: user 1.69 s, sys: 2.58 s, total: 4.27 s
Wall time: 5.49 s


## Isolate hashtags

In [47]:
%%time
# df_channels['hashtags'] = df_channels['message_no_mentions'].str.findall("#[A-Za-z0-9_]+")
df_channels['hashtags'] = df_channels['message_no_mentions'].apply(lambda x: {word.strip(",").strip(".") for word in x.split() if word.startswith("#")})

CPU times: user 8.28 s, sys: 1.33 s, total: 9.61 s
Wall time: 10.4 s


In [48]:
df_channels[['hashtags', 'message_no_mentions']].sample(10)

,hashtags,message_no_mentions
1339153,"{#Украина, #Россия, #Белоруссия}",На повестке снова Светлана Тихановская. На это...
303670,{},Вау! Мне предложили рекламу размещать у себя. ...
1472075,{},"Евгений Рылов — первый, но ставит планку себе ..."
1271839,{},Какое небо голубое... ‍ ѣѣ источник
897397,{},"Господи, Хабаровский след в Бейруте"
1522199,{},Жителей вблизи Ходынского бульвара избавят от ...
1543638,{},Председатель Патриотической ассоциации Донбасс...
1120040,{},Оппозиционер Алексей Навальный опубликовал на ...
1535100,{},‼ Владимир Путин и Реджеп Эрдоган в ходе встре...
1037127,{},🪖На этой неделе сразу несколько региональных п...


In [49]:
df_channels[['hashtags', 'message_no_mentions']].message_no_mentions.iloc[52089]

'Зеленский про погибшую в Первомайске женщину: ‘у нас есть одна проблема....’  Ужас какой!'

In [50]:
def cut_hasgtags(set_of_hashtags, text):
    for word in list(set_of_hashtags):
        text = text.replace(word, '')
    return text.strip()

In [51]:
df_channels['message_no_hashtags'] = df_channels.apply(lambda x: cut_hasgtags(x['hashtags'], x['message_no_mentions']), axis=1)

## Remove everything except letters and punctuation

In [52]:
from string import punctuation

punctuation_minimal = '!(),-.:;?%'

cyrillic_letters = u"абвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ"
latin_letters_numbers = "1234567890abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ "
allowed_symbols = cyrillic_letters+latin_letters_numbers + punctuation_minimal


In [53]:
def clean_text(string, allowed_symbols=allowed_symbols):
    getVals = list(filter(lambda x: x in allowed_symbols, string))
    result = "".join(getVals)
    result = re.sub(' +', ' ', result)

    return result

In [54]:
%%time
df_channels['message_clean'] = df_channels['message_no_hashtags'].apply(lambda x: clean_text(x))

CPU times: user 46.9 s, sys: 2.31 s, total: 49.2 s
Wall time: 51.5 s


In [55]:
df_channels['message_clean'].sample(10).iloc[0]

'Следственный комитет горда Москвы - несуществующий орган. Дальше можно не читать. В Москве действует Главное следственное управление по городу Москве Следственного комитета России. Ситуация, когда дураку поручили фейк нарисовать, а он даже списать с шапки бланка не смог правильное название органа.'

In [56]:
print(f' We have {len(df_channels[df_channels["message_clean"]==""])} ({100*len(df_channels[df_channels["message_clean"]==""])/len(df_channels):2f}%) empty clean messages')

 We have 11355 (0.925792%) empty clean messages


In [57]:
print(f' We have {len(df_channels[df_channels["message_clean"]!=""])} ({100*len(df_channels[df_channels["message_clean"]!=""])/len(df_channels):2f}%) meaningful clean messages')

 We have 1215162 (99.074208%) meaningful clean messages


In [91]:
only_letters = cyrillic_letters+latin_letters_numbers

In [96]:
%%time
df_channels['message_words_only_lower'] = df_channels['message_clean'].apply(lambda x: clean_text(x, allowed_symbols=only_letters).lower())

CPU times: user 47.8 s, sys: 2.64 s, total: 50.4 s
Wall time: 57.1 s


In [102]:
df_channels['message_words_only_lower'].sample(10).iloc[0]

'ну андрюха это при свидетелях говорил да и в сети часто фельдмана подкалывал но я не говорил что не нравится просто любил его притролить'

# Approaches

## Spacy RU model

In [1]:
import spacy
from spacy.lang.ru.examples import sentences

In [ ]:
#python3 -m spacy download ru_core_news_md

In [475]:
# spacy.util.set_data_path("/Users/katerynaburovova/PycharmProjects/dehumanization/lib/python3.10/site-packages/ru_core_news_md")

In [2]:
nlp = spacy.load('ru_core_news_md')

In [81]:
def tokenize_spacy(text):
    doc = nlp(text)
    result = []
    for token in doc:
        result.append([token.text, token.pos_, token.dep_, token.lemma_, token.morph])
    return result

In [82]:
tokenize_spacy(sentences[0])

[['Apple',
  'PROPN',
  'nsubj',
  'apple',
  Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing],
 ['рассматривает',
  'VERB',
  'ROOT',
  'рассматривать',
  Aspect=Imp|Mood=Ind|Number=Sing|Person=Third|Tense=Pres|VerbForm=Fin|Voice=Act],
 ['возможность',
  'NOUN',
  'obj',
  'возможность',
  Animacy=Inan|Case=Acc|Gender=Fem|Number=Sing],
 ['покупки',
  'NOUN',
  'nmod',
  'покупка',
  Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing],
 ['стартапа',
  'NOUN',
  'nmod',
  'стартап',
  Animacy=Inan|Case=Gen|Gender=Masc|Number=Sing],
 ['из', 'ADP', 'case', 'из', ],
 ['Соединённого',
  'ADJ',
  'amod',
  'соединённого',
  Case=Gen|Degree=Pos|Gender=Neut|Number=Sing],
 ['Королевства',
  'PROPN',
  'nmod',
  'королевство',
  Animacy=Inan|Case=Gen|Gender=Neut|Number=Sing],
 ['за', 'ADP', 'case', 'за', ],
 ['$', 'SYM', 'nmod', '$', ],
 ['1', 'NUM', 'appos', '1', ],
 ['млрд',
  'NOUN',
  'nmod',
  'млрд',
  Animacy=Inan|Case=Gen|Gender=Masc|Number=Sing]]

## RAZDEL tokenization

Handles "... - ... " as one word (even for composite ones)

In [226]:
# from razdel import tokenize
# def tokenize_razdel(text):
#     return(list(tokenize(text)))

## spacy_russian_tokenizer

https://github.com/aatimofeev/spacy_russian_tokenizer

In [342]:
# from spacy.lang.ru import Russian
# from spacy_russian_tokenizer import RussianTokenizer, MERGE_PATTERNS
# text = "Не ветер, а какой-то ураган!"
# nlp = Russian()
# doc = nlp(text)
# russian_tokenizer = RussianTokenizer(nlp, MERGE_PATTERNS)
# nlp.add_pipe(russian_tokenizer, name='russian_tokenizer')
# doc = nlp(text)
# print([token.text for token in doc])

TypeError: add() takes exactly 2 positional arguments (339 given)

## lang-uk

[не сабворд](https://github.com/lang-uk/ner-uk/blob/master/doc/tokenization.md)

In [83]:
# from tokenize_uk import tokenize_words
# tokenize_words(text)


## stanza


In [84]:
# import stanza
# stanza.download('ru')


In [85]:
# nlp_ru = stanza.Pipeline('ru', processors='tokenize')

In [86]:
# def get_stanza_tokens(text):
#     doc = nlp_ru(text)
#     word_tokens = [token.text for sent in doc.sentences for token in sent.tokens]
#     # result = [word for word in word_tokens if word not in stop_words_ru]
#     return word_tokens

## Subword (BPE?) BERT

Subword, but weird NER

In [87]:
# from transformers import AutoTokenizer, AutoModel

In [88]:
# tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
# model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")

In [308]:
# # pip install transformers sentencepiece
# import torch
# from transformers import AutoTokenizer, AutoModel
# tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
# model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")
# # model.cuda()  # uncomment it if you have a GPU
#
# def embed_bert_cls(text, model, tokenizer):
#     t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
#     with torch.no_grad():
#         model_output = model(**{k: v.to(model.device) for k, v in t.items()})
#     embeddings = model_output.last_hidden_state[:, 0, :]
#     embeddings = torch.nn.functional.normalize(embeddings)
#     return embeddings[0].cpu().numpy()
#
# print(embed_bert_cls('привет мир', model, tokenizer).shape)
# # (312,)

In [348]:
# def tokens_bert_cls(text, model, tokenizer):
#     encoding = tokenizer.encode(text)
#     return tokenizer.convert_ids_to_tokens(encoding)
#
# tokens_bert_cls(test_string, model, tokenizer)
# # (312,)

['[CLS]',
 '[UNK]',
 '11',
 '-',
 'го',
 'полка',
 'гон',
 '##яют',
 'танк',
 '[UNK]',
 '11',
 'полк',
 'Н',
 '##М',
 'ДНР',
 'продолжает',
 'наносить',
 'удары',
 'по',
 'противнику',
 'в',
 'Первомай',
 '##ском',
 '.',
 'На',
 'видео',
 'танковый',
 'батальон',
 'под',
 'командованием',
 'Севера',
 'бьет',
 'по',
 'позициям',
 'ВСУ',
 ',',
 'заодно',
 'заставляя',
 'мета',
 '##ться',
 'и',
 'спешно',
 'удалять',
 '##ся',
 'появи',
 '##вшийся',
 'на',
 'свою',
 'беду',
 'украинский',
 'танк',
 '.',
 '@',
 'war',
 '##gon',
 '##zo',
 '*',
 'наш',
 'проект',
 'существует',
 'на',
 'средства',
 'подписчиков',
 ',',
 'карта',
 'для',
 'помощи',
 '427',
 '##9',
 '380',
 '##6',
 '984',
 '##2',
 '952',
 '##1',
 '[SEP]']

In [349]:
# tokens_bert_cls(text, model, tokenizer)

['[CLS]', 'Не', 'ветер', ',', 'а', 'какой', '-', 'то', 'ураган', '!', '[SEP]']

## Execution time

In [ ]:
# df_test_sample = df_channels.sample(100000)

In [353]:
# %%time
# df_test_sample['spacy_tokens'] = df_test_sample['message'].apply(lambda x: tokenize_spacy(x))

CPU times: user 19.5 s, sys: 1.02 s, total: 20.5 s
Wall time: 21.1 s


In [354]:
# %%time
# df_test_sample['bert_tokens'] = df_test_sample['message'].apply(lambda x: tokens_bert_cls(x, model, tokenizer))

CPU times: user 13 s, sys: 331 ms, total: 13.3 s
Wall time: 13.5 s


In [355]:
# %%time
# df_test_sample['lang_uk_tokens'] = df_test_sample['message'].apply(lambda x: tokenize_words(x))

CPU times: user 1.91 s, sys: 75.3 ms, total: 1.99 s
Wall time: 2.02 s


In [356]:
# %%time
# df_test_sample['razdel_tokens'] = df_test_sample['message'].apply(lambda x: tokenize_razdel(x))

CPU times: user 13.8 s, sys: 2.08 s, total: 15.9 s
Wall time: 16.9 s


In [374]:
# %%time
# df_test_sample['stanza_tokens'] = df_test_sample['message'].sample(100).apply(lambda x: get_stanza_tokens(x))

CPU times: user 2.81 s, sys: 234 ms, total: 3.05 s
Wall time: 3.11 s


In [375]:
# %%time
# df_test_sample['stanza_tokens'] = df_test_sample['message'].apply(lambda x: get_stanza_tokens(x))

CPU times: user 40min 18s, sys: 2min 44s, total: 43min 2s
Wall time: 45min


In [384]:
# from random import randrange
#
# df_channels.message.iloc[randrange(len(df_channels))]

'Отправка по электронной почте 2_5325905093675979275.pdf'

In [401]:
# rnd_text = df_channels.message.iloc[randrange(len(df_channels))]
# print(rnd_text)
#
# tokenize_spacy(rnd_text)

Сообщается о раненых в рядах турецких сил в результате подрыва мины на трассе М4 в провинции Идлиб. Турецкие вертолеты направляются к месту взрыва для эвакуации раненых.


[['Сообщается', 'VERB', 'ROOT', 'сообщаться'],
 ['о', 'ADP', 'case', 'о'],
 ['раненых', 'NOUN', 'obl', 'раненый'],
 ['в', 'ADP', 'case', 'в'],
 ['рядах', 'NOUN', 'nmod', 'ряд'],
 ['турецких', 'ADJ', 'amod', 'турецкий'],
 ['сил', 'NOUN', 'nmod', 'сила'],
 ['в', 'ADP', 'case', 'в'],
 ['результате', 'NOUN', 'obl', 'результат'],
 ['подрыва', 'NOUN', 'nmod', 'подрыв'],
 ['мины', 'NOUN', 'nmod', 'мина'],
 ['на', 'ADP', 'case', 'на'],
 ['трассе', 'NOUN', 'nmod', 'трасса'],
 ['М4', 'PROPN', 'appos', 'м4'],
 ['в', 'ADP', 'case', 'в'],
 ['провинции', 'NOUN', 'nmod', 'провинция'],
 ['Идлиб', 'PROPN', 'appos', 'идлиб'],
 ['.', 'PUNCT', 'punct', '.'],
 ['Турецкие', 'ADJ', 'amod', 'турецкий'],
 ['вертолеты', 'NOUN', 'nsubj', 'вертолёт'],
 ['направляются', 'VERB', 'ROOT', 'направляться'],
 ['к', 'ADP', 'case', 'к'],
 ['месту', 'NOUN', 'obl', 'место'],
 ['взрыва', 'NOUN', 'nmod', 'взрыв'],
 ['для', 'ADP', 'case', 'для'],
 ['эвакуации', 'NOUN', 'nmod', 'эвакуация'],
 ['раненых', 'NOUN', 'nmod', 'ранены

# Tokenization and lemmatization

In [89]:
def tokenize_spacy(text):
    doc = nlp(text)
    result = []
    for token in doc:
        result.append([token.text, token.pos_, token.dep_, token.lemma_, token.morph])
    return result

In [114]:
def get_lemmas(text):
    doc = nlp(text)
    result = []
    for token in doc:
        result.append([token.lemma_])
    return result

In [104]:
df_sample = df_channels.sample(1000)

In [105]:
%%time
df_sample['tokens'] = df_sample['message_words_only_lower'].apply(lambda x: tokenize_spacy(x))

CPU times: user 16.7 s, sys: 352 ms, total: 17 s
Wall time: 18 s


In [108]:
df_sample['message_words_only_lower'].iloc[536]

'президент россии владимир путин наградил бориса грызлова орденом за заслуги перед отечеством i степени орден вручен за выдающиеся заслуги в укреплении российской государственности и реализации внешнеполитического курса страны также основанием для награды послужили его значительный вклад в развитие обороннопромышленного комплекса россии и многолетняя добросовестная работа'

In [109]:
df_sample['tokens'].iloc[536]

[['президент',
  'NOUN',
  'nsubj',
  'президент',
  Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing],
 ['россии',
  'PROPN',
  'nmod',
  'россия',
  Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing],
 ['владимир',
  'PROPN',
  'appos',
  'владимир',
  Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing],
 ['путин',
  'PROPN',
  'flat:name',
  'путин',
  Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing],
 ['наградил',
  'VERB',
  'ROOT',
  'наградить',
  Aspect=Perf|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act],
 ['бориса',
  'PROPN',
  'obj',
  'борис',
  Animacy=Anim|Case=Acc|Gender=Masc|Number=Sing],
 ['грызлова',
  'PROPN',
  'flat:name',
  'грызлов',
  Animacy=Anim|Case=Acc|Gender=Masc|Number=Sing],
 ['орденом',
  'NOUN',
  'xcomp',
  'орден',
  Animacy=Inan|Case=Ins|Gender=Masc|Number=Sing],
 ['за', 'ADP', 'case', 'за', ],
 ['заслуги',
  'NOUN',
  'obl',
  'заслуга',
  Animacy=Inan|Case=Acc|Gender=Fem|Number=Plur],
 ['перед', 'ADP', 'case', 'перед', ],
 ['отечеством',
  'N

In [115]:
%%time
df_sample['lemmas'] = df_sample['message_words_only_lower'].apply(lambda x: get_lemmas(x))

CPU times: user 16.8 s, sys: 433 ms, total: 17.2 s
Wall time: 19.1 s


In [126]:
df_sample['tokens'].iloc[536]


[['как', 'SCONJ', 'mark', 'как', ],
 ['показало',
  'VERB',
  'parataxis',
  'показать',
  Aspect=Perf|Gender=Neut|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act],
 ['исследование',
  'NOUN',
  'nsubj',
  'исследование',
  Animacy=Inan|Case=Nom|Gender=Neut|Number=Sing],
 ['statista', 'X', 'appos', 'statista', Foreign=Yes],
 ['global', 'X', 'flat:foreign', 'global', Foreign=Yes],
 ['consumer', 'X', 'flat:foreign', 'consumer', Foreign=Yes],
 ['survey', 'X', 'flat:foreign', 'survey', Foreign=Yes],
 ['почти', 'ADV', 'advmod', 'почти', Degree=Pos],
 ['каждый', 'DET', 'det', 'каждый', Case=Nom|Gender=Masc|Number=Sing],
 ['пятый',
  'ADJ',
  'amod',
  'пятый',
  Case=Nom|Degree=Pos|Gender=Masc|Number=Sing],
 ['респондент',
  'NOUN',
  'nsubj',
  'респондент',
  Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing],
 ['из', 'ADP', 'case', 'из', ],
 ['великобритании',
  'PROPN',
  'nmod',
  'великобритания',
  Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing],
 ['18', 'NUM', 'appos', '18', ],
 ['о

In [124]:
df_sample['lemmas'].iloc[45]

[['как'],
 ['показать'],
 ['исследование'],
 ['statista'],
 ['global'],
 ['consumer'],
 ['survey'],
 ['почти'],
 ['каждый'],
 ['пятый'],
 ['респондент'],
 ['из'],
 ['великобритания'],
 ['18'],
 ['обычно'],
 ['страдать'],
 ['от'],
 ['похмелие'],
 ['на'],
 ['следующий'],
 ['утро'],
 ['после'],
 ['приём'],
 ['алкоголь'],
 ['такой'],
 ['образ'],
 ['британец'],
 ['оказаться'],
 ['наиболее'],
 ['подверженный'],
 ['этот'],
 ['недуг'],
 ['среди'],
 ['житель'],
 ['восемь'],
 ['страна'],
 ['участвовать'],
 ['в'],
 ['исследование']]

In [125]:
df_sample['message_words_only_lower'].iloc[56]

'блумберг администрация байдена подталкивает союзников ес к завершению разработки широкого пакета санкций против российских банков и энергетических компаний которые могут быть введены совместно с сша если кремль нападет на украину некоторые потенциальные шаги из списка такие как отключение россии от платежной системы swift считаются крайне проблематичными сказали собеседники агентства ссылаясь на потенциальное нарушение работы мировых рынков энергоносителей и других товаров российского экспорта еще одним предметом беспокойства является перспектива того что путин может прекратить экспорт газа россия обеспечивает около 40 поставок в европу в сухом остатке план что и говорить был превосходный простой и ясный лучше не придумать недостаток у него был только один было совершенно неизвестно как привести его в исполнение'

In [127]:
def get_lemma_vec(text):
    doc = nlp(text)
    result = []
    for token in doc:
        result.append([token.lemma])
    return result

In [141]:
tokenize_spacy('хохляцкого')


[['хохляцкого',
  'ADJ',
  'ROOT',
  'хохляцкого',
  Animacy=Anim|Case=Acc|Degree=Pos|Gender=Masc|Number=Sing]]

# OOV handling

https://spacy.io/usage/processing-pipelines#custom-components-user-hooks

should we add a hook that returns zero vectors for OOV terms?

In [135]:
%%time
df_channels['lemmas'] = df_channels['message_words_only_lower'].apply(lambda x: get_lemmas(x))


KeyboardInterrupt



In [136]:
%%time
df_channels['tokens'] = df_channels['message_words_only_lower'].apply(lambda x: tokenize_spacy(x))

KeyboardInterrupt: 

In [137]:
df_channels

,id,date,views,reactions,to_id,fwd_from,message,type,duration,channel_name,...,message_no_emoji,url_list,message_no_urls,mentions,message_no_mentions,quotes,hashtags,message_no_hashtags,message_clean,message_words_only_lower
0,12347,2022-12-15 16:32:15+00:00,9914.0,MessageReactions(results=[ReactionCount(reacti...,PeerChannel(channel_id=1183570279),NaN,А трансляция тем временем идет. Подписывайтесь...,photo,NaN,mardanaka,...,А трансляция тем временем идет. Подписывайтесь...,[www.youtube.com],А трансляция тем временем идет. Подписывайтесь,[],А трансляция тем временем идет. Подписывайтесь,[],{},А трансляция тем временем идет. Подписывайтесь,А трансляция тем временем идет. Подписывайтесь,а трансляция тем временем идет подписывайтесь
1,12346,2022-12-15 15:00:03+00:00,29207.0,MessageReactions(results=[ReactionCount(reacti...,PeerChannel(channel_id=1183570279),NaN,"В России ещё есть те, кто способен возрождать ...",photo,NaN,mardanaka,...,"В России ещё есть те, кто способен возрождать ...",[],"В России ещё есть те, кто способен возрождать ...",[],"В России ещё есть те, кто способен возрождать ...",[МЭФ. Экономика для людей],{},"В России ещё есть те, кто способен возрождать ...","В России ещё есть те, кто способен возрождать ...",в россии ещё есть те кто способен возрождать н...
2,12345,2022-12-15 14:21:22+00:00,41058.0,MessageReactions(results=[ReactionCount(reacti...,PeerChannel(channel_id=1183570279),NaN,«Катаргейт» набирает обороты - Обвиняемая в ко...,text,NaN,mardanaka,...,«Катаргейт» набирает обороты - Обвиняемая в ко...,[],«Катаргейт» набирает обороты - Обвиняемая в ко...,[],"""Катаргейт"" набирает обороты - Обвиняемая в ко...",[Катаргейт],{},"""Катаргейт"" набирает обороты - Обвиняемая в ко...",Катаргейт набирает обороты - Обвиняемая в корр...,катаргейт набирает обороты обвиняемая в корруп...
3,12344,2022-12-15 13:08:35+00:00,40696.0,MessageReactions(results=[ReactionCount(reacti...,PeerChannel(channel_id=1183570279),"MessageFwdHeader(date=datetime.datetime(2022, ...",🔥В гостях у @Metametrica побывали товарищи из ...,photo,NaN,mardanaka,...,В гостях у @Metametrica побывали товарищи из ...,[],В гостях у @Metametrica побывали товарищи из ...,"[Metametrica, Metametrica]",В гостях у побывали товарищи из украинского п...,[Богдана Хмельницкого],{},В гостях у побывали товарищи из украинского п...,В гостях у побывали товарищи из украинского пр...,в гостях у побывали товарищи из украинского пр...
4,12343,2022-12-15 12:31:23+00:00,51690.0,MessageReactions(results=[ReactionCount(reacti...,PeerChannel(channel_id=1183570279),NaN,Глава Минобороны Грузии назвал грузинских наем...,text,NaN,mardanaka,...,Глава Минобороны Грузии назвал грузинских наем...,[],Глава Минобороны Грузии назвал грузинских наем...,[],Глава Минобороны Грузии назвал грузинских наем...,"[Есть случаи, когда едут воевать за гонорар, в...",{},Глава Минобороны Грузии назвал грузинских наем...,Глава Минобороны Грузии назвал грузинских наем...,глава минобороны грузии назвал грузинских наем...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1607921,7,2019-09-28 10:13:52+00:00,3190.0,NaN,PeerChannel(channel_id=1253974160),NaN,Бот для связи: @obrazbuduschego2_bot Донастрое...,text,NaN,obrazbuduschego2,...,Бот для связи: @obrazbuduschego2_bot Донастрое...,[],Бот для связи: @obrazbuduschego2_bot Донастрое...,"[obrazbuduschego2_bot, protonmail]","Бот для связи: Донастроен, работает, если кто...",[],{},"Бот для связи: Донастроен, работает, если кто...","Бот для связи: Донастроен, работает, если кто-...",бот для связи донастроен работает если ктото н...
1607922,6,2019-09-28 09:35:07+00:00,3239.0,NaN,PeerChannel(channel_id=1253974160),NaN,"Печальная весть о Марке Захарове, человеке, ко...",text,NaN,obrazbuduschego2,...,"Печальная весть о Марке Захарове, человеке, ко...",[],"Печальная весть о Марке Захарове, человеке, ко...",[],"Печальная весть о Марке Захарове, человеке, ко...","[Чай, кофе и другие колониальные товары]",{},"Печальная весть о Мар

In [144]:
df_channels.message_words_only_lower.iloc[33330]

'и посмотрите елену серветтаз она там местами срывает покровы про альбац браудера и так далее и вообще она очень хорошая'

In [ ]:
#pymorphy2
#sparknlp
#на чому тренувався spacy - взяти з цього корпусу словник
#взятисловник
#службові частини мови забрати